In [1]:
import ase
from ase import io
from ase.io.espresso import write_espresso_in
import numpy as np
import os
from ase.visualize import view

In [ ]:
# Read the slab file into an ASE Atoms object
slab = ase.io.read('../zrb2-slab-001/qe-calcs/relax/zrb2-slab-001-relax.out', index=-1)
view(slab)

<Popen: returncode: None args: ['/home/cganley2/.conda/envs/ds/bin/python', ...>

In [3]:
# Input setting for Quantum ESPRESSO
pseudopotentials = {'Zr': 'Zr.pbe-spn-kjpaw_psl.1.0.0.UPF',
                    'B' : 'B.pbe-n-kjpaw_psl.1.0.0.UPF',
                    'O' : 'O.pbe-n-kjpaw_psl.0.1.UPF'}
input_data = {
    'scf': {
        'calculation': 'scf',
        'outdir': './tmp',
        'pseudo_dir': '/home/cganley2/qe-data/pseudopotentials/SSSP_1.3.0_PBE_precision', # will need to update this--do you have this directory somewhere?
        'tprnfor': True,
        'tstress': True,
        'ecutwfc': 50,
        'ecutrho': 400,
        'electron_maxstep': 200,
        'occupations': 'smearing',
        'conv_thr': 1e-7,
        'mixing_mode': 'local-TF',
        'ion_dynamics': 'bfgs',
    },
    'relax': {
        'calculation': 'relax',
        'outdir': './tmp',
        'pseudo_dir': '/home/cganley2/qe-data/pseudopotentials/SSSP_1.3.0_PBE_precision', # same as above
        'tprnfor': True,
        'tstress': True,
        'ecutwfc': 50,
        'ecutrho': 400,
        'electron_maxstep': 200,
        'occupations': 'smearing',
        'conv_thr': 1e-7,
        'mixing_mode': 'local-TF',
        'ion_dynamics': 'bfgs',
    }
}
kpts = (2, 2, 1)

In [ ]:
# iterate over unique z-values in slab file
# randomly change up to 4 atoms to O
# create input file for SCF and relax
# need to consider output directory parsing

In [3]:
unique_z_vals = np.unique(np.round(slab.get_positions(), 1)[:, 2])
unique_z_vals

array([10. , 11.8, 13.5, 15.3, 17.1, 18.9])

In [ ]:
for z_val in unique_z_vals:
    # find the indices of the atoms in the slab that have this z value
    layer_indices = np.array([])
    for index, xyz_coords in enumerate(slab.get_positions()):
        if np.round(xyz_coords[2], 1) == z_val:
            # print('The rounded z coordinate matches the z value in the loop. The atom index is {0} and is atom {1}'.format(index, slab.get_chemical_symbols()[index]))
            layer_indices = np.append(layer_indices, index)
    
    # identify 4 random indices from the layer to use for defecting
    defect_indices = np.random.shuffle(layer_indices)[:4]

    # substitute an O at each index, save the structure and QE input file
    for index in defect_indices:
        symbols = slab.get_chemical_symbols()
        # during testing you can use `view(slab)` (or whatever you rename the modified structure to) to see what the object looks like in a 3d rendering
        # can also save separate as .xyz, .extxyz, or .cif file for downloading and visualization on local computer; io.write(filename='filename.extension', images=slab, format='xyz')
        # i think you can use the os module to make each 'layerN-oM/scf' and 'layerN-oM/relax' folder, where N is the layer number (lowest z-val first) and M is the number of O's added to that layer
        # symbols[index] = 'O'
        # write_espresso_in('zrb2-001/qe-calcs/layerN-oM/scf/pw.in', slab, input_data=input_data['scf'], pseudopotentials=pseudopotentials, kpts=kpts)
        # write_espresso_in('zrb2-001/qe-calcs/layerN-oM/relax/pw.in', slab, input_data=input_data['relax'], pseudopotentials=pseudopotentials, kpts=kpts)


[ 4.  7. 20. 23. 36. 39. 52. 55.]
[ 1.  2. 17. 18. 33. 34. 49. 50.]
[ 5.  6. 21. 22. 37. 38. 53. 54.]
[ 0.  3. 16. 19. 32. 35. 48. 51.]
[12. 15. 28. 31. 44. 47. 60. 63.]
[ 9. 10. 25. 26. 41. 42. 57. 58.]
[13. 14. 29. 30. 45. 46. 61. 62.]
[ 8. 11. 24. 27. 40. 43. 56. 59.]
